# Setup

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

In [ ]:
# Clone `CreateDebateScraper` library from github
!git clone https://github.com/utkarsh512/CreateDebateScraper.git
%cd CreateDebateScraper/src/nested/

In [ ]:
!pip install transformers

In [ ]:
from   copy                     import deepcopy
# import cpnet
from   itertools                import accumulate
import json
from   matplotlib               import pyplot as plt
import networkx as nx
import nltk
import numpy as np
import pandas as pd
import pickle
import re
from   scipy                    import stats
import textwrap
from   thread                   import Comment, Thread
from   tqdm                     import tqdm
nltk.download('punkt') # For tokenizers
nltk.download('stopwords')
import matplotlib
from   nltk.tokenize            import TweetTokenizer
from   nltk.corpus              import stopwords
from   pprint                   import pprint
from   transformers             import BertModel, BertTokenizer
# import shifterator as sh
# import wordcloud
# import skbio
matplotlib.rcParams.update({'font.size': 18})
matplotlib.rcParams["figure.figsize"] = (12, 5)
STOP_WORDS = list(stopwords.words('english'))

In [ ]:
# Custom routine to clean texts scraped from Web.
# It removes hyperlinks, punctuation marks (except apostrophe)

tknz = TweetTokenizer()

def clean_text(text):
    """
    Preprocessing text
    """
    text = text.lower()
    text = re.sub(r"http\S+", "", text)
    text = re.sub(r"www\S+", "", text)
    text = re.sub("-", " ", text)
    text = re.sub("\s+", " ", text)
    text = re.sub("\u2018", "X", text) 
    text = re.sub("\u2019", "X", text) 
    text = re.sub("\'", "X", text) 
    wordTokens_ = tknz.tokenize(text)
    wordTokens = list()
    for x in wordTokens_:
        x = ''.join([v for v in x if v.isalnum() or v == ' '])
        if len(x) > 0 and x != 'X':
            x = x.replace('X', '\'')
            wordTokens.append(x)
    return wordTokens

In [ ]:
comments = dict()

# Topical forums on CreateDebate. We have scraped comments for all of the
# following forurm.
categories = ['business', 'comedy', 'entertainment', 'health', 'law', 'nsfw',
              'politics2', 'religion', 'science', 'shopping', 'sports',
              'technology', 'travel', 'world']

# However, we will be analyzing comments from selected forum only!
# These forum have at least 10k comments each.
categories_selected = ['politics2', 'religion', 'world', 
                       'science', 'law', 'technology']

for x in categories_selected:
    comments[x] = list()

In [ ]:
# Loading comments from select forums

for cat in tqdm(categories_selected):
    fp = open('/content/gdrive/MyDrive/DL/CreateDebate/' + cat + '/threads.log', 'rb')

    # Get all the `Thread` objects pickled while scraping.
    threads = list()
    try:
        while True:
            e = pickle.load(fp)
            threads.append(e)
    except EOFError:
        fp.close()

    # While classifying CreateDebate comments, we used comments as per author mode.
    # Hence, using the same mode to attach classification score with the comments.
    # 
    # score < 0.5 -> ad hominem comment
    #       > 0.5 -> non ad hominem comment
    authors = dict()
    for thread in threads:
        for k, v in thread.comments.items():
            try:
                authors[v.author].append((v, k))
            except:
                authors[v.author] = list()
                authors[v.author].append((v, k))

    ctr = 0
    # Load the classification score of the comments.
    with open('/content/gdrive/MyDrive/DL/CreateDebate/' + cat + '/comments_with_score.log', 'rb') as fp:
        cws = pickle.load(fp)
    # Attach classification score with the comments.
    for author in authors.keys():
        for i in range(len(authors[author])):
            comment, cid = authors[author][i]
            foo = deepcopy(comment.__dict__)
            foo['tag'] = cat
            foo['score'] = cws[ctr][0]
            foo['validation'] = cws[ctr][1][0]
            foo['id'] = int(cid[3:])
            comments[cat].append(foo)
            ctr += 1

# Analysis

In [ ]:
# Partition the dataset into polar and non-polar comments

polar_cids = set()

comments_p = dict()
comments_np = dict()

for x in categories_selected:
    comments_p[x] = list()
    comments_np[x] = list()

    for comment in comments[x]:
        if comment['polarity'] == 'Not Available':
            comments_np[x].append(deepcopy(comment))
        else:
            comments_p[x].append(deepcopy(comment))

In [ ]:
for x in categories_selected:
    print(f'{x} - {len(comments_p[x])} - {len(comments_np[x])}')

In [ ]:
# User characterisitics

user_list_p = dict()
user_list_np = dict()

for cat in categories_selected:
    user_list_p[cat] = set()
    for comment in comments_p[cat]:
        user_list_p[cat].add(comment['author'])

    user_list_np[cat] = set()
    for comment in comments_np[cat]:
        user_list_np[cat].add(comment['author'])


In [ ]:
for cat in categories_selected:
    print(f'{cat} - {len(user_list_p[cat])} - {len(user_list_np[cat])} - {len(user_list_p[cat] & user_list_np[cat])}')

## User profile characteristics

In [ ]:
# Loading CreateDebate profile characteristics into dataframe
df = pd.read_json('/content/gdrive/MyDrive/DL/CreateDebate/profile/results.json', lines=True)

# Extract useful characteristics
reward_points_map = {k : v for k, v in zip(df['username'].tolist(), df['reward_points'].tolist())}
efficiency_map    = {k : v for k, v in zip(df['username'].tolist(), df['efficiency'].tolist())}
allies_map        = {k : len(v) for k, v in zip(df['username'].tolist(), df['allies'].tolist())}
enemies_map       = {k : len(v) for k, v in zip(df['username'].tolist(), df['enemies'].tolist())}
hostiles_map      = {k : len(v) for k, v in zip(df['username'].tolist(), df['hostiles'].tolist())}

In [ ]:
def profile_characteristics_stats(user_subset):
    """
    Returns average and standard deviation of profile characteristics for 
    given subset of users.

    :param user_subset: Iterable containing usernames

    >>> avgs, stds = profile_characterisitics_stat(user_subset)
    >>> rewards_avg, efficiency_avg, n_allies_avg, n_enemies_avg, n_hostiles_avg = avgs
    >>> rewards_std, efficiency_std, n_allies_std, n_enemies_std, n_hostiles_std = stds

    Note that profile characteristics for some users might not be present in our
    dataset as some users might have deleted their account when we scraped the
    forum to obtain these characteristics.
    """
    rewards_ = list()
    efficiency_ = list()
    n_allies = list()
    n_enemies = list()
    n_hostiles = list()

    for user in user_subset:
        try:
            rewards_.append(reward_points_map[user])
        except:pass
        try:
            efficiency_.append(efficiency_map[user])
        except:pass
        try:
            n_allies.append(allies_map[user])
        except:pass
        try:
            n_enemies.append(enemies_map[user])
        except:pass
        try:
            n_hostiles.append(hostiles_map[user])
        except:pass
    
    grpd_data = [rewards_, efficiency_, n_allies, n_enemies, n_hostiles]
    avgs = [np.average(x) for x in grpd_data]
    stds = [np.std(x) for x in grpd_data]
    
    return avgs, stds

In [ ]:
def display_stats(user_subset):
    user_chr_avg, user_chr_std = profile_characteristics_stats(user_subset) 
    print('Reward points: %.2f ± %.2f' % (user_chr_avg[0], user_chr_std[0]))
    print('Efficiency   : %.2f ± %.2f' % (user_chr_avg[1], user_chr_std[1]))
    print('# Allies     : %.2f ± %.2f' % (user_chr_avg[2], user_chr_std[2]))
    print('# Enemies    : %.2f ± %.2f' % (user_chr_avg[3], user_chr_std[3]))
    print('# Hostiles   : %.2f ± %.2f' % (user_chr_avg[4], user_chr_std[4]))

In [ ]:
for cat in categories_selected: 
    print('\n\n-----------------------------\n\n')
    print(f'Category: {cat}\n\n')
    print('Users writing polar comments:\n')
    display_stats(user_list_p[cat])
    print('\n\nUsers writing non-polar comments:\n')
    display_stats(user_list_np[cat])

## Slur words analysis

In [ ]:
polar_comments_full_text = dict()
nonpolar_comments_full_text = dict()

for cat in tqdm(categories_selected):
    polar_comments_full_text[cat] = list()
    nonpolar_comments_full_text[cat] = list()

    for comment in comments_p[cat]:
        polar_comments_full_text[cat].append(' '.join(clean_text(comment['body'].strip())))
    
    for comment in comments_np[cat]:
        nonpolar_comments_full_text[cat].append(' '.join(clean_text(comment['body'].strip())))

    polar_comments_full_text[cat] = ' '.join(polar_comments_full_text[cat])
    nonpolar_comments_full_text[cat] = ' '.join(nonpolar_comments_full_text[cat])

In [ ]:
SLUR_WORDS = {
  "jews": [
    "jews",
    "oven dodger",
    "nazi",
    "dirty jew",
    "holocaust",
    "kikesucker",
    "hook nose",
    "kike"
  ],
  "homosexual": [
    "faggots usually",
    "fucking queer",
    "the biggest faggot",
    "dyke",
    "you fucking faggot",
    "hate faggots",
    "queer",
    "homosexual",
    "the faggots",
    "faggot",
    "faggots usually have",
    "gay",
    "faggots",
    "dykey",
    "ugly dyke",
    "faggots like you",
    "you a fag",
    "lesbian",
    "homo",
    "is a faggot",
    "like a faggot",
    "dykes",
    "faggots like",
    "faggot if you ever"
  ],
  "women": [
    "ugly dyke",
    "woman terrorist",
    "nigress",
    "bitch",
    "slut",
    "women",
    "sheeboon",
    "negress",
    "mud shark",
    "women threat",
    "you a lame bitch",
    "your a cunt",
    "white bitch",
    "niggeress",
    "hoe",
    "dykes",
    "niggress",
    "sheboon",
    "feminazi"
  ],
  "blacks": [
    "pavement ape",
    "the niggers",
    "negress",
    "porch monkey",
    "that nigger",
    "this nigger",
    "sheboon",
    "all niggers",
    "eurafrica",
    "shut up nigger",
    "picaninny",
    "african attack",
    "spearchucker",
    "how many niggers",
    "nigger",
    "africa",
    "niggers are in my",
    "dindu nuffin",
    "stupid nigger",
    "moolie",
    "niggers",
    "bluegum",
    "nigger ass",
    "you niggers",
    "fucking nigger",
    "nigger music",
    "niggress",
    "you a nigger",
    "many niggers are",
    "nigress",
    "blacks",
    "teenaper",
    "sheeboon",
    "dumb nigger",
    "niggeress",
    "pickaninny",
    "nigga"
  ],
  "muslim": [
    "muslim immigrant",
    "islam",
    "mudslime",
    "mooslem",
    "muslim refugee",
    "musslime",
    "shitlam",
    "muslim invasion",
    "moslime",
    "mooslamic",
    "muzzie",
    "allah akbar",
    "mooslime",
    "musloid",
    "mudslimes",
    "muslim",
    "muslimes",
    "moslum",
    "mussie",
    "muzrat",
    "muslim countries",
    "muzzy",
    "moslim",
    "jihadi",
    "muslim country",
    "moslem",
    "muzzrat",
    "mooslim"
  ],
  "arabs": [
    "towel head",
    "goatfucker",
    "arabs",
    "goathumper",
    "raghead",
    "rag head",
    "goathumping",
    "towelhead",
    "camel jockey",
    "sandnigger",
    "camel fucker",
    "sand nigger"
  ],
  "generic": [
    "to rape",
    "raped and",
    "shithole country",
    "get raped",
    "raped",
    "is a fucking",
    "shit skin",
    "raped by",
    "hate you",
    "fake empowerment",
    "abusive women",
    "fuck you too",
    "violence",
    "wit a lame nigga",
    "they all look",
    "alllivesmatter",
    "shithole countries",
    "fucking hate",
    "trailer trash",
    "kill all",
    "terrorist threat",
    "harassment",
    "kill yourself",
    "shitskin",
    "okay to be white",
    "fucking hate you"
  ],
  "white": [
    "full of white",
    "white trash",
    "white devil",
    "white",
    "are all white",
    "white boy",
    "white ass",
    "white bitch",
    "hillbilly",
    "whigger",
    "white christian",
    "white person",
    "all white",
    "white nigger",
    "redneck",
    "white honky",
    "wigger",
    "them white"
  ],
  "economy": [
    "ghetto"
  ],
  "immigrant": [
    "illegal immigrants",
    "immigrant not welcome",
    "immigrant terror",
    "mexcrement",
    "go back to where you come from",
    "muslim refugee",
    "illegal aliens",
    "refugee",
    "protect from immigrants",
    "negro",
    "refugees",
    "immigrant",
    "refugee invasion",
    "go back to where they come from",
    "refugees impact",
    "bring ebola",
    "immigrants",
    "illegal alien",
    "immigrant invasion",
    "bring disease"
  ],
  "mental": [
    "retard",
    "mongoloid",
    "retarded"
  ],
  "asians": [
    "asians",
    "ching chong",
    "chinaman"
  ]
}

In [ ]:
polar_slur_count = dict()
nonpolar_slur_count = dict()

for cat in categories_selected:
    polar_slur_count[cat] = dict()
    nonpolar_slur_count[cat] = dict()

    polar_token_count = len(polar_comments_full_text[cat].split())
    nonpolar_token_count = len(nonpolar_comments_full_text[cat].split())

    for slur_group in SLUR_WORDS.keys():
        polar_slur_count[cat][slur_group] = 0
        nonpolar_slur_count[cat][slur_group] = 0

        for slur_words in SLUR_WORDS[slur_group]:
            polar_slur_count[cat][slur_group] += polar_comments_full_text[cat].count(slur_words) * len(slur_words.split())
            nonpolar_slur_count[cat][slur_group] += nonpolar_comments_full_text[cat].count(slur_words) * len(slur_words.split())
        
        polar_slur_count[cat][slur_group] /= polar_token_count
        nonpolar_slur_count[cat][slur_group] /= nonpolar_token_count

In [ ]:
def plot_slur_words(cat, saveas):
    x = list(SLUR_WORDS.keys())
    y1 = [] # Polar
    y2 = [] # Non-polar

    for slur_group in x:
        y1.append(polar_slur_count[cat][slur_group])
        y2.append(nonpolar_slur_count[cat][slur_group])

    ticks = np.arange(len(x))
    width = 0.30

    fig, ax = plt.subplots()
    fig.set_size_inches(15, 12)
    subplot1 = ax.bar(ticks - width / 2, y1, width, label='Polar', tick_label=x)
    subplot2 = ax.bar(ticks + width / 2, y2, width, label='Non-polar', tick_label=x)

    ax.set_ylabel('Probability')
    ax.set_title(f'Probability of being a slur word')
    ax.set_xticks(ticks)
    ax.set_xticklabels(x, rotation=45, ha='right')
    ax.legend()
    plt.savefig(saveas)

In [ ]:
plot_slur_words('politics2', '/content/gdrive/MyDrive/Temp/MTPImages/politics_slur_prob_1.pdf')

In [ ]:
plot_slur_words('religion', '/content/gdrive/MyDrive/Temp/MTPImages/religion_slur_prob_1.pdf')

In [ ]:
plot_slur_words('world')

In [ ]:
plot_slur_words('science')

In [ ]:
plot_slur_words('law')

In [ ]:
plot_slur_words('technology')

## Triggers

In [ ]:
comments_p['politics2'][0]

In [ ]:
# TODO: Get top 100 ad hominem comments from polar and non-polar content
# and visualize them using attention scores

comment_score_p = dict()
comment_score_np = dict()

for cat in categories_selected:
    comment_score_p[cat] = list()
    comment_score_np[cat] = list()

    for comment in comments_p[cat]:
        comment_score_p[cat].append((comment['id'], 1 - comment['score']))
    
    for comment in comments_np[cat]: 
        comment_score_np[cat].append((comment['id'], 1 - comment['score']))
    
    comment_score_p[cat] = sorted(comment_score_p[cat], key=lambda x: x[1], reverse=True)
    comment_score_np[cat] = sorted(comment_score_np[cat], key=lambda x: x[1], reverse=True)

In [ ]:
top_ah_comments_p = dict()
top_ah_comments_np = dict()

for cat in categories_selected:
    top_ah_comments_p[cat] = list()
    top_ah_comments_np[cat] = list()

    top_cid_p = list()
    top_cid_np = list()

    for i in range(100):
        top_cid_p.append(comment_score_p[cat][i][0])
        top_cid_np.append(comment_score_np[cat][i][0])
    
    top_cid_p = set(top_cid_p)
    top_cid_np = set(top_cid_np)

    for comment in comments_p[cat]: 
        if comment['id'] in top_cid_p:
            top_ah_comments_p[cat].append(' '.join(clean_text(comment['body'])))
    
    for comment in comments_np[cat]: 
        if comment['id'] in top_cid_np: 
            top_ah_comments_np[cat].append(' '.join(clean_text(comment['body'])))

    print(len(top_ah_comments_p[cat]), len(top_ah_comments_np[cat]))

In [ ]:
class Visualizer:
    """Wrapper for creating heatmaps for documents"""
    def __init__(self):
        self._header = r'''\documentclass[10pt,a4paper]{article}
\usepackage[left=1.00cm, right=1.00cm, top=1.00cm, bottom=2.00cm]{geometry}
\usepackage{color}
\usepackage{tcolorbox}
\usepackage{CJK}
\usepackage{adjustbox}
\tcbset{width=0.9\textwidth,boxrule=0pt,colback=red,arc=0pt,auto outer arc,left=0pt,right=0pt,boxsep=5pt}
\begin{document}
\begin{CJK*}{UTF8}{gbsn}''' + '\n\n'

        self._footer = r'''\end{CJK*}
\end{document}'''

    def visualize(self,
                  word_list,
                  attention_list,
                  label_list,
                  latex_file,
                  title,
                  batch_size=20,
                  color='blue'):
        """Routine to generate attention heatmaps for given texts
        ---------------------------------------------------------
        Input:
        :param word_list: list of texts (each text is a list of words)
        :param attention_list: scores for each word, dimension same as word_list
        :param label_list: label for each text
        :param latex_file: name of the latex file
        :param title: title of latex file
        :param batch_size: Number of comments in each batch
        :param color: color used for visualization, can be 'blue', 'red', 'green', etc.
        """
        word_list_processed = []
        for x in word_list:
            word_list_processed.append(self._clean_word(x))

        with open(latex_file, 'w', encoding='utf-8') as f:
            f.write(self._header)
            f.write('\\section{%s}\n\n' % title)

            n_examples = len(word_list)
            n_batches = n_examples // batch_size

            for i in range(n_batches):
                batch_word_list = word_list_processed[i * batch_size: (i + 1) * batch_size]
                batch_attention_list = attention_list[i * batch_size: (i + 1) * batch_size]
                batch_label_list = label_list[i * batch_size: (i + 1) * batch_size]
                f.write('\\subsection{Batch %d}\n\n' % (i + 1))
                for j in range(batch_size):
                    f.write('\\subsubsection{Comment %d - %s}\n\n' % (j + 1, batch_label_list[j]))
                    sentence = batch_word_list[j]
                    score = batch_attention_list[j]
                    assert len(sentence) == len(score)
                    f.write('\\noindent')
                    for k in range(len(sentence)):
                        f.write('\\colorbox{%s!%s}{' % (color, score[k]) + '\\strut ' + sentence[k] + '} ')
                    f.write('\n\n')

            f.write(self._footer)

    @staticmethod
    def _clean_word(word_list):
        new_word_list = []
        for word in word_list:
            for latex_sensitive in ["\\", "%", "&", "^", "#", "_", "{", "}"]:
                if latex_sensitive in word:
                    word = word.replace(latex_sensitive, '\\' + latex_sensitive)
            new_word_list.append(word)
        return new_word_list

In [ ]:
model_version = '/content/gdrive/MyDrive/DL/cnerg-bert-adhominem'
do_lower_case = True
model = BertModel.from_pretrained(model_version, output_attentions=True)
tokenizer = BertTokenizer.from_pretrained(model_version, do_lower_case=do_lower_case)

In [ ]:
INTENSITY = 70

def attention_scores(text, layers=None, heads=None):
    sentence_a = text
    inputs = tokenizer.encode_plus(sentence_a, None, return_tensors='pt', add_special_tokens=True)
    input_ids = inputs['input_ids']
    attention = model(input_ids)[-1]
    input_id_list = input_ids[0].tolist() # Batch index 0
    tokens = tokenizer.convert_ids_to_tokens(input_id_list) 
    sz = len(tokens)
    matrix = [0 for j in range(sz)]
    if layers is None:
        layers = [x for x in range(12)]
    if heads is None:
        heads = [x for x in range(12)]
    for layer in layers:
        for head in heads:
            for j in range(sz):
                matrix[j] += attention[layer][0, head, 0, j].item()
    for j in range(sz):
        matrix[j] = (matrix[j]) / (len(layers) * len(heads))
    return (tokens, matrix)

In [ ]:
def clean_array(w, a):
    W = []
    A = []
    for i in range(len(w)):
        if (w[i].startswith('##')):
            W[len(W) - 1] += w[i][2:]
            A[len(A) - 1] = (A[len(A) - 1] + a[i]) / 2
        else:
            W.append(w[i])
            A.append(a[i])
    return clean_apos(W, A)

def clean_apos(w, a):
    W = []
    A = []
    ctr = 0
    while ctr != len(w):
        if w[ctr] == '\'':
            W[-1] += w[ctr] + w[ctr + 1]
            A[-1] = min(INTENSITY, A[-1] + a[ctr] + a[ctr + 1])
            ctr += 2
        else:
            W.append(w[ctr])
            A.append(a[ctr])
            ctr += 1
    return W, A

In [ ]:
def top_three_tokens(text):
    words, attentions = attention_scores(text)
    words = words[1:-1] # Remove start and end tags
    attentions = attentions[1:-1]
    assert len(words) == len(attentions)
    words, attentions = clean_array(words, attentions)
    assert len(words) == len(attentions)
    top_tokens = list()
    for i in range(len(words)):
        top_tokens.append((attentions[i], i))
    top_tokens = sorted(top_tokens, reverse=True)
    ind = [0]
    cur = 1
    while len(ind) < 3:
        take = True
        for ids in ind:
            take = take and abs(top_tokens[ids][1] - top_tokens[cur][1]) > 2
        if take:
            ind.append(cur)
        cur += 1
    xx = []
    for x in ind:
        xx.append(top_tokens[x][1])
    scores = [0 for i in range(len(words))]
    for w in xx:
        lst = [w - 1, w, w + 1]
        for j in lst:
            if j >= 0 and j < len(words):
                scores[j] = INTENSITY
    return words, scores

In [ ]:
viz = Visualizer()

def create_latex_file(cat, do_polar=True, filename='sample.tex'):
    top_ah_comments = top_ah_comments_p[cat] if do_polar else top_ah_comments_np[cat]
    words_list = list()
    scores_list = list()

    for comment in top_ah_comments:
        try:
            words, scores = top_three_tokens(comment)
        except:
            continue
        words_list.append(words)
        scores_list.append(scores)
    
    label = 'Polar' if do_polar else 'Non-polar'
    labels_list = [label for _ in range(len(words_list))]
    
    viz.visualize(words_list, scores_list, labels_list,
                  latex_file=filename,
                  title=f'{label} comments in {cat} forum',
                  batch_size=len(words_list),
                  color='cyan')

In [ ]:
category_ = 'world'

In [ ]:
create_latex_file(category_, True)

In [ ]:
create_latex_file(category_, False)